# [TR-006] Interactive 3D plots

In [ ]:
%%sh
pip install ipywidgets==7.6.5 matplotlib==3.4.3 numpy==1.19.5 sympy==1.8 > /dev/null

<!-- cspell:ignore cstride descrip displaystyle facecolor ianhi ipyslider ipywidget ipywidgets mplot rstride toolitems valinit valmax valmin valstep -->

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib widget
import os

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ipywidgets import widgets as ipywidgets
from matplotlib import cm
from matplotlib import widgets as mpl_widgets

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

This report illustrates how to interact with [`matplotlib`](https://matplotlib.org) 3D plots through [Matplotlib sliders](https://matplotlib.org/stable/api/widgets_api.html) and [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html). This might be implemented later on in {mod}`symplot` and/or [`mpl_interactions`](https://mpl-interactions.readthedocs.io) (see {issue}`ianhi/mpl-interactions#89`).

In this example, we create a surface plot (see {obj}`~mpl_toolkits.mplot3d.Axes3D.plot_surface`) for the following function.

In [ ]:
x, y, a, b = sp.symbols("x y a b")
expression = sp.sqrt(x ** a + sp.sin(y / b) ** 2)
expression

sqrt(x**a + sin(y/b)**2)

The function is formulated with {mod}`sympy`, but we use {func}`~sympy.utilities.lambdify.lambdify` to express it as a {mod}`numpy` function.

In [ ]:
numpy_function = sp.lambdify(
    args=(x, y, a, b),
    expr=expression,
    modules="numpy",
)

A surface plot has to be generated over a {func}`numpy.meshgrid`. This defines the $xy$-plane over which we want to plot our function.

In [ ]:
x_min, x_max = 0.1, 2
y_min, y_max = -50, +50
x_values = np.linspace(x_min, x_max, num=20)
y_values = np.linspace(y_min, y_max, num=40)
X, Y = np.meshgrid(x_values, y_values)

The $z$-values for {obj}`~mpl_toolkits.mplot3d.Axes3D.plot_surface` can now be simply computed as follows:

In [ ]:
a_init = -0.5
b_init = 20
Z = numpy_function(X, Y, a=a_init, b=b_init)

We now want to create sliders for $a$ and $b$, so that we can live-update the surface plot through those sliders.

## Matplotlib widgets

Matplotlib provides its own way to define {mod}`matplotlib.widgets`.

In [ ]:
fig1, ax1 = plt.subplots(ncols=1, subplot_kw={"projection": "3d"})

# Create sliders and insert them within the figure
plt.subplots_adjust(bottom=0.25)
a_slider = mpl_widgets.Slider(
    ax=plt.axes([0.2, 0.1, 0.65, 0.03]),
    label=f"${sp.latex(a)}$",
    valmin=-2,
    valmax=2,
    valinit=a_init,
)
b_slider = mpl_widgets.Slider(
    ax=plt.axes([0.2, 0.05, 0.65, 0.03]),
    label=f"${sp.latex(b)}$",
    valmin=10,
    valmax=50,
    valinit=b_init,
    valstep=1,
)


# Define what to do when a slider changes
def update_plot(val=None):
    a = a_slider.val
    b = b_slider.val
    ax1.clear()
    Z = numpy_function(X, Y, a, b)
    ax1.plot_surface(
        X,
        Y,
        Z,
        rstride=3,
        cstride=1,
        cmap=cm.coolwarm,
        antialiased=False,
    )
    ax1.set_xlabel(f"${sp.latex(x)}$")
    ax1.set_ylabel(f"${sp.latex(y)}$")
    ax1.set_zlabel(f"${sp.latex(expression)}$")
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_zticks([])
    ax1.set_facecolor("white")
    fig1.canvas.draw_idle()


a_slider.on_changed(update_plot)
b_slider.on_changed(update_plot)

# Plot the surface as initialization
update_plot()
plt.show()

{{ run_interactive }}

![interactive_output](006-matplotlib-slider.gif)

## `ipywidgets`

As an alternative, you can use [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html). This package has lot more sliders to offer than Matplotlib, and they look nicer, but it only work within a Jupyter notebook.

For more info, see [Using Interact](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html).

### Using `interact`

Simplest option is to use the [`ipywidgets.interact()`](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html) function:

In [ ]:
fig2, ax2 = plt.subplots(ncols=1, subplot_kw={"projection": "3d"})


@ipywidgets.interact(a=(-2.0, 2.0), b=(10, 50))
def plot2(a=a_init, b=b_init):
    ax2.clear()
    Z = numpy_function(X, Y, a, b)
    ax2.plot_surface(
        X,
        Y,
        Z,
        rstride=3,
        cstride=1,
        cmap=cm.coolwarm,
        antialiased=False,
    )
    ax2.set_xlabel(f"${sp.latex(x)}$")
    ax2.set_ylabel(f"${sp.latex(y)}$")
    ax2.set_zlabel(f"${sp.latex(expression)}$")
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_zticks([])
    ax2.set_facecolor("white")
    fig2.canvas.draw_idle()

{{ run_interactive }}

![interactive_output](006-ipywidgets-slider.svg)

### Using `interactive_output`

You can have more control with [`ipywidgets.interactive_output()`](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html). That allows defining the sliders independently, so that you can arrange them as a user interface:

In [ ]:
fig3, ax3 = plt.subplots(ncols=1, subplot_kw={"projection": "3d"})
a_ipyslider = ipywidgets.FloatSlider(
    description=f"${sp.latex(a)}$",
    value=a_init,
    min=-2,
    max=2,
    step=0.1,
    readout_format=".1f",
)
b_ipyslider = ipywidgets.IntSlider(
    description=f"${sp.latex(b)}$",
    value=b_init,
    min=10,
    max=50,
)


def plot3(a=a_init, b=b_init):
    ax3.clear()
    Z = numpy_function(X, Y, a, b)
    ax3.plot_surface(
        X,
        Y,
        Z,
        rstride=3,
        cstride=1,
        cmap=cm.coolwarm,
        antialiased=False,
    )
    ax3.set_xlabel(f"${sp.latex(x)}$")
    ax3.set_ylabel(f"${sp.latex(y)}$")
    ax3.set_zlabel(f"${sp.latex(expression)}$")
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax3.set_zticks([])
    ax3.set_facecolor("white")
    fig3.canvas.draw_idle()


ui = ipywidgets.HBox([a_ipyslider, b_ipyslider])
output = ipywidgets.interactive_output(
    plot3, controls={"a": a_ipyslider, "b": b_ipyslider}
)
display(ui, output)

{{ run_interactive }}

![interactive_output](006-ipywidget-interactive_output.gif)